# Gradient Boosting Regressor Notebook
### Author: Krzysztof Chmielewski

This notebook implements a gradient boosting regressor from scratch and evaluates it on the Boston housing dataset. It demonstrates:
- Sequential tree building with residual fitting
- Gradient boosting algorithm (additive ensemble, shrinkage)
- Regression tree integration for boosting
- Training and evaluation (RMSE, MAE, R² metrics)

## Model Purpose and Applications

**Gradient Boosting** is a powerful ensemble algorithm that sequentially builds weak learners (typically shallow trees), each correcting the residual errors of all previous trees. It often achieves state-of-the-art performance on regression and classification problems.

### Key Use Cases:
- **Kaggle competitions**: Often the top-performing model for structured data
- **Housing price prediction**: High-accuracy real estate valuation
- **Demand forecasting**: Predicting product demand with complex patterns
- **Risk modeling**: Financial and credit risk assessment
- **Click-through rate (CTR) prediction**: Online advertising performance prediction
- **Ranking problems**: Learning-to-rank for search engines
- **Medical prognosis**: Patient outcome prediction with high accuracy
- **Energy consumption forecasting**: Complex demand patterns

### Strengths:
- **Highest accuracy** — often achieves state-of-the-art results on structured data
- **Sequential learning** — each tree corrects previous errors (residuals)
- **Robust to outliers** — iterative error correction handles anomalies
- Handles complex non-linear relationships
- Can work with missing data (with modifications)
- Provides feature importance rankings
- Flexible loss functions (MSE, MAE, custom)

### Limitations:
- **Requires careful tuning** — many hyperparameters (learning rate, n_trees, max_depth)
- **Slow to train** — sequential nature prevents parallelization across trees
- Prone to **overfitting** if not carefully regulated
- Difficult to interpret compared to single trees
- Sensitive to learning rate and stopping criteria
- Computationally expensive for very large datasets

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils.evaluation import eval_regression

In [2]:
def MSE(y):
    if len(y) == 0: return 0
    return np.mean((y - np.mean(y)) ** 2)

def split_dataset(X, y, feature, threshold):
    left_mask = X[:, feature] <= threshold
    right_mask = X[:, feature] > threshold
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

## Gradient Boosting: Core Concepts

Gradient Boosting builds an additive ensemble by iteratively fitting weak learners (regression trees) to the **residuals** of the previous models.

### Initial Prediction

Start with an initial regression tree:

$$\hat{y}^{(0)} = \text{Tree}_0(X)$$

### Sequential Residual Fitting

For each iteration $m = 1, 2, \ldots, M$:

1. Compute residuals (errors) of current predictions:
   $$r_m = y - \hat{y}^{(m-1)}$$

2. Fit a new tree to these residuals:
   $$\text{Tree}_m = \text{fit}(X, r_m)$$

3. Update predictions with shrinkage (learning rate $\eta$):
   $$\hat{y}^{(m)} = \hat{y}^{(m-1)} + \eta \cdot \text{Tree}_m(X)$$

The learning rate $\eta$ (typically 0.01-0.2) controls the contribution of each new tree, preventing overfitting.

### Final Prediction

$$\hat{y} = \text{Tree}_0(X) + \eta \cdot \sum_{m=1}^{M} \text{Tree}_m(X)$$

This additive ensemble approach allows Gradient Boosting to iteratively reduce prediction error, often achieving superior performance.

In [3]:
class RegressionTree():

    def __init__(self, max_depth = 5, min_samples_split = 2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    class Node:
        def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value = value 

    def __best_split(self, X, y):
        best_feature = None
        best_threshold = None
        best_mse = np.inf

        _, n_features = X.shape

        for feature in range(n_features):
            thresholds = np.unique(X[:,feature])

            for threshold in thresholds:
                _, y_left, _, y_right = split_dataset(X,y,feature,threshold)

                if len(y_left) == 0 or len(y_right) == 0: continue

                mse_left = MSE(y_left)
                mse_right = MSE(y_right)

                weighted_mse = (len(y_left)/len(y) * mse_left + len(y_right)/len(y) * mse_right)

                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_mse
    
    def __build_tree(self, X, y, depth=0):
        if depth >= self.max_depth or len(y) < self.min_samples_split:
            return self.Node(value=np.mean(y))
        
        feature, threshold, mse = self.__best_split(X,y)

        if feature is None or mse >= MSE(y):
            return self.Node(value=np.mean(y))
        
        X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)

        left_child = self.__build_tree(X_left, y_left, depth+1)
        right_child = self.__build_tree(X_right, y_right, depth+1)

        return self.Node(feature, threshold, left_child, right_child)
    
    def __predict_sample(self, x, node: Node):
        if node.left is None and node.right is None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.__predict_sample(x, node.left)
        else:
            return self.__predict_sample(x, node.right)
        
    def fit(self, X, y):
        self.root = self.__build_tree(X, y)

    def predict(self, X):
        return np.array([self.__predict_sample(x, self.root) for x in X])


In [25]:
class GradientBoost():
    def __init__(self, lr=0.01, n_trees=10, max_depth=5, min_samples_split=2):
        self.lr = lr
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.init_tree = RegressionTree(
            max_depth=self.max_depth,
            min_samples_split=self.min_samples_split
        )
        self.trees = []
    
    def fit(self, X, y, verbose=True):
        self.init_tree.fit(X,y)
        y_pred = self.init_tree.predict(X)

        for i in range(self.n_trees):
            residuals = y - y_pred
            new_tree = RegressionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            new_tree.fit(X, residuals)
            y_pred += self.lr * new_tree.predict(X)

            self.trees.append(new_tree)

            if verbose and i % (self.n_trees // 10) == 0:
                loss = np.sqrt(np.mean((y - y_pred)**2))
                print(f"Tree {i}, RMSE: {loss:.4f}")

    def predict(self, X):
        pred = self.init_tree.predict(X)
        for tree in self.trees:
            pred += self.lr * tree.predict(X)
        return pred

## Boston Housing Dataset

The Boston housing dataset contains information about housing in the Boston area with 506 samples and 13 features. The target variable `MEDV` is the median home value in $1000s. This dataset is commonly used for regression benchmarking.

In [5]:
data = pd.read_csv('data/boston/boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B-1000,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


## Training

Split the data into training and test sets and build the gradient boosting ensemble.

- Train/test split: `train_test_split(X, Y, test_size=0.3, random_state=42)` for standard regression split
- Hyperparameters: `lr=0.2` (learning rate), `n_trees=50` (number of boosting iterations), `max_depth=5` (max tree depth)
- Each tree is trained on residuals from previous predictions
- Sequential nature: each tree corrects errors of the ensemble so far

In [6]:
X = data.drop("MEDV", axis=1).values
Y = data["MEDV"].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [26]:
GB = GradientBoost(lr=0.2, n_trees=50)
GB.fit(X_train, y_train)

Tree 0, RMSE: 2.3348
Tree 5, RMSE: 1.8439
Tree 10, RMSE: 1.5207
Tree 15, RMSE: 1.2754
Tree 20, RMSE: 1.0589
Tree 25, RMSE: 0.9180
Tree 30, RMSE: 0.7403
Tree 35, RMSE: 0.6342
Tree 40, RMSE: 0.5365
Tree 45, RMSE: 0.4678


## Evaluation

After training, we predict continuous values on the test set and evaluate using regression metrics:

- **RMSE (Root Mean Squared Error)**: $\sqrt{\frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)^2}$ — penalizes large errors
- **MAE (Mean Absolute Error)**: $\frac{1}{n} \sum_{i=1}^{n} |\hat{y}_i - y_i|$ — robust to outliers
- **R² Score**: $1 - \frac{\sum (\hat{y}_i - y_i)^2}{\sum (y_i - \bar{y})^2}$ — proportion of variance explained (0 to 1)

We use the local helper `eval_regression(y_true, y_pred)` to compute and display these metrics.

In [27]:
y_pred = GB.predict(X_test)
eval_regression(y_test, y_pred);

MSE: 13.237
RMSE: 3.638
MAE: 2.429
R2: 0.822
